In [9]:
import boto3
import sagemaker
from sagemaker import get_execution_role, image_uris
from sagemaker.sklearn.model import SKLearnModel

# Assume that you've already saved your model to a file named 'model.pkl'
s3_model_path = 's3://namthai-bigdata-bucket/model/pipeline.tar.gz'


# retrive image uri
image_uri = image_uris.retrieve(framework='sklearn', region='ap-southeast-1',
                                version='1.2-1',
                                image_scope='inference')

# endpoint name
endpoint_name = 'v-twac-2024'


# Get the execution role for SageMaker
role = get_execution_role()



# Create a SageMaker Scikit-learn Model
sklearn_model = SKLearnModel(model_data=s3_model_path,
                             role=role,
                             image_uri=image_uri,
                             entry_point='inference.py',
                             dependencies=['requirements.txt', 'transformer.py'])
                             
                             

# Deploy the model to a real-time endpoint
predictor = sklearn_model.deploy(initial_instance_count=1,
                         instance_type='ml.m5.xlarge',
                         endpoint_name=endpoint_name,
                         logs=True)

-----------------------------------------*

UnexpectedStatusException: Error hosting endpoint sagemaker-scikit-learn-2024-01-15-09-09-12-159: Failed. Reason: The primary container for production variant AllTraffic did not pass the ping health check. Please check CloudWatch logs for this endpoint.. Try changing the instance type or reference the troubleshooting page https://docs.aws.amazon.com/sagemaker/latest/dg/async-inference-troubleshooting.html

In [12]:
# Remove the endpoint
predictor.delete_endpoint(delete_endpoint_config=True)

sagemaker = boto3.client('sagemaker')

sagemaker.delete_endpoint(EndpointName='v-twac-2024')

{'ResponseMetadata': {'RequestId': 'a42f5119-b708-4550-92c2-34a524b41299',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': 'a42f5119-b708-4550-92c2-34a524b41299',
   'content-type': 'application/x-amz-json-1.1',
   'content-length': '0',
   'date': 'Mon, 15 Jan 2024 09:35:31 GMT'},
  'RetryAttempts': 0}}